In [1]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [3]:

options = Options()

options.add_argument('--user-data-dir=/Users/Administrator/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 1')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--start-maximized")

options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

# Add a preference to disable image loading


capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True




In [4]:
file_path = 'urls.txt'

categories = []

with open(file_path, 'r') as file:
    for line in file:     
        category = line.strip()
        categories.append(category)
file_path = 'cities.txt'

cities = []

with open(file_path, 'r') as file:
    for line in file:     
        cityy = line.strip().lower()
        cities.append(cityy)
categories

['https://dir.indiamart.com/impcat/pet-bottle-scrap.html',
 'https://dir.indiamart.com/impcat/pp-scrap.html',
 'https://dir.indiamart.com/impcat/hdpe-scrap.html',
 'https://dir.indiamart.com/impcat/ld-plastic-scrap.html',
 'https://dir.indiamart.com/impcat/hdpe-drum-scrap.html',
 'https://dir.indiamart.com/impcat/waste-plastic.html',
 'https://dir.indiamart.com/impcat/pvc-scrap.html',
 'https://dir.indiamart.com/impcat/ldpe-scrap.html',
 'https://dir.indiamart.com/impcat/pet-bottle-flake.html']

In [43]:
all_details = {}
index = 0
urls = []
names = []
history = 1
for count,category in enumerate(categories):
    driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
    for city in cities:
        if history%12==0 :
            driver.close()
            time.sleep(10)
            driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
            history += 1
        else:
            history += 1
        try:
            wb_address = category + "?grid_view=1"
            wb_address = wb_address.replace("impcat",city)
            driver.get(wb_address)
            
            time.sleep(2)
            html_text = driver.page_source
            soup = BeautifulSoup(html_text,"lxml")
            
            # If captcha appears
            while "check the box below to proceed" in html_text:
                html_text = driver.page_source
                soup = BeautifulSoup(html_text,"lxml")
                continue

            if "No results found for" in html_text:
                continue


            soup_old = ""
            print(f'Category Number:{count+1}/{len(categories)}----Dict_Length:{len(all_details)}',end='\r')
            while True:


                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                driver.implicitly_wait(3)

                try:

                    btn = driver.find_element(By.CLASS_NAME,"fm2")
                    driver.execute_script("arguments[0].click();",btn)

                except:

                    break
            html_text = driver.page_source
            soup = BeautifulSoup(html_text,"lxml")
            products = soup.find("div",class_="q_hm1 cnhdr fxmn")
            products = products.find("div",class_='lay-left')
            products = products.find("ul",class_="mListGrp w100 sid_df fww wlm mFrgn")
            sections = products.find_all("li")
            filtered_sections = []

            for section in sections:
                if section.has_attr('id'):
                    filtered_sections.append(section)

            for section in filtered_sections:
                details={}
                try:
                    name = section.find("h3").text
                    price = section.find("p").text
                    try:
                        url = section.find('li',class_="mListPrc").find("a")["href"]
                    except:
                        try:
                            url = section.find("a")["href"]
                        except:
                            url = None
                        
                except:
                    continue

                details["name"]= name
                details["price"] = price
                details["url"] = url
                details["category"] = category.split("/")[-1].split(".html")[0]


                all_details[index] = details
                index+=1
        except:
            continue

    driver.close()

In [44]:
df = pd.DataFrame().from_dict(all_details).T
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.to_csv("indiamart.csv")
df

,name,price,url,category
0,Grinded Green PET Bottle Scrap,₹58/ Kg,https://www.indiamart.com/proddetail/green-pet...,pet-bottle-scrap
1,Baled Pet Bottle Scrap - Natural Colour,₹40/ Kg,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-scrap
2,Multicolor PET Bottle Scrap,₹12/ Kg,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-scrap
3,"Aluminium Bottle Caps Scrap, 25 Kg",₹65/ Kg,https://www.indiamart.com/proddetail/aluminium...,pet-bottle-scrap
4,"Pet Bottle Scrap, Pack Size: Bags",₹30/ Kilogram,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-scrap
...,...,...,...,...
11941,Mix Crushed Pet Bottle Scrap,₹50/ Kg,,pet-bottle-flake
11942,Flakes,,https://www.indiamart.com/kalra-agencies-yamun...,pet-bottle-flake
11943,Pet Flakes,,https://www.indiamart.com/million-internationa...,pet-bottle-flake
11944,Pet Flakes,,https://www.indiamart.com/rana-plastic-industr...,pet-bottle-flake


In [31]:
df = pd.read_csv("indiamart.csv")
df.head()

,Unnamed: 0,name,price,url,category
0,0,Grinded Green PET Bottle Scrap,₹58/ Kg,https://www.indiamart.com/proddetail/green-pet...,pet-bottle-scrap
1,1,Baled Pet Bottle Scrap - Natural Colour,₹40/ Kg,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-scrap
2,2,Multicolor PET Bottle Scrap,₹12/ Kg,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-scrap
3,3,"Aluminium Bottle Caps Scrap, 25 Kg",₹65/ Kg,https://www.indiamart.com/proddetail/aluminium...,pet-bottle-scrap
4,4,"Pet Bottle Scrap, Pack Size: Bags",₹30/ Kilogram,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-scrap


In [32]:
df["price"] = df["price"].apply(lambda x: str(x).strip(" "))
df["price"] = df["price"].apply(lambda x: str(x).lower())

df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kg","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("KG","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram(s)","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("KILOGRAMS","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kgs","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kilogram","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kg(s)","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("per kg","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kgs","kg"))

df["price"] = df["price"].apply(lambda x: str(x).replace("kg onwards","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram Onwards","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("TONNE","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Tonne","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("metric ton(s)","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("ton","tonne") if str(x).split("/")[-1]==" ton" else x)
df["price"] = df["price"].apply(lambda x: str(x).replace("metric ton","tonne"))

df["price"] = df["price"].apply(lambda x: str(x).replace("Ton","tonne"))



In [33]:
df["metric"] = df["price"].apply(lambda x: str(x).split("/")[-1].strip())
df["metric"].value_counts().to_dict()

{'nan': 81,
 'kg': 51,
 'plastic bottle scrap': 3,
 'pet content:  96- 100%': 2,
 'pet plastic bottle scraps': 2,
 'tonne': 1,
 'capacity:  1 l': 1,
 'waste product of bottles': 1,
 'bottle plastic scrap': 1,
 'scrap bailed pet plastic bottles': 1,
 'bottle scrap': 1,
 'application:  plastic industry': 1,
 'pet content:  > 95%': 1,
 'green bottel wast': 1,
 'pet content:  90%': 1,
 'pet bottle scrab': 1,
 'preform weight:  11-15 gms': 1,
 'ubc cans scrap': 1,
 'pet bottles scrap': 1,
 'pet content:  90- 95%': 1}

In [42]:
df[df["metric"]=="pet content:  90- 95%"]["url"][152]

'https://www.indiamart.com/proddetail/pet-scrap-20769283012.html'

In [19]:
names_to_filter = ["kg", "tonne", "metric"]
filtered_df = df[df['metric'].isin(names_to_filter)]
filtered_df = filtered_df.reset_index(drop=True)

In [21]:
filtered_df.to_csv("filtered_indiamart.csv",index=False)

In [22]:
filtered_df

,name,price,url,category,metric
0,Inconel 690 Scrap,₹550/ kg,https://www.indiamart.com/proddetail/inconel-6...,Industrial Scrap,kg
1,Titanium Scrap,₹250/ kg,https://www.indiamart.com/proddetail/titanium-...,Industrial Scrap,kg
2,"Stainless Steel Plate Cut Scrap, 50 KG, Packag...",₹100/ kg,https://www.indiamart.com/proddetail/stainless...,Industrial Scrap,kg
3,"Black Mild Steel Scrap, For Metal Industry, Pa...",₹43/ kg,https://www.indiamart.com/proddetail/mild-stee...,Industrial Scrap,kg
4,"Aluminium Scrap Lithium Battery, Model Name/Nu...",₹90/ kg,https://www.indiamart.com/proddetail/scrap-lit...,Industrial Scrap,kg
...,...,...,...,...,...
3494,"Dull White Polyester Monofilament Yarn, For Se...",₹165/ kg,https://www.indiamart.com/proddetail/polyester...,Recycled Yarn,kg
3495,Cotton Yarn,₹110/ kg,https://www.indiamart.com/proddetail/cotton-ya...,Recycled Yarn,kg
3496,100% Egyptian Cotton Yarn,₹410/ kg,https://www.indiamart.com/proddetail/100-egypt...,Recycled Yarn,kg
3497,"Dull White Polyester Spun Yarn, For Weaving",₹100/ kg,https://www.indiamart.com/proddetail/polyester...,Recycled Yarn,kg
